# Exploring Image Data

Starting with the simple sample mcap examples

In [4]:
import rerun as rr

IMAGE_PATH = "/Users/nick/Downloads/image.rrd"

In [6]:
rec = rr.dataframe.load_recording(IMAGE_PATH)
print(
    f"Recording: {rec.application_id()}, {rec.recording_id()}"
)

Recording: mcap-image-example.mcap, rec_c17995bfc533480a83980e305f0e4885


In [8]:
# Mostly a note to future me but it'd be nice if this actually printed out something.
print(rec)

In [7]:
print(rec.schema())

Column name: /camera/annotations:circles
	Entity path: /camera/annotations
	Archetype: foxglove.ImageAnnotations
	Component type: 
	Component: circles
Column name: /camera/annotations:points
	Entity path: /camera/annotations
	Archetype: foxglove.ImageAnnotations
	Component type: 
	Component: points
Column name: /camera/annotations:texts
	Entity path: /camera/annotations
	Archetype: foxglove.ImageAnnotations
	Component type: 
	Component: texts
Column name: /camera/annotations:TextLog:level
	Entity path: /camera/annotations
	Archetype: rerun.archetypes.TextLog
	Component type: rerun.components.TextLogLevel
	Component: TextLog:level
Column name: /camera/annotations:TextLog:text
	Entity path: /camera/annotations
	Archetype: rerun.archetypes.TextLog
	Component type: rerun.components.Text
	Component: TextLog:text
Column name: /camera/annotations:id
	Entity path: /camera/annotations
	Archetype: rerun.mcap.Channel
	Component type: 
	Component: id
Column name: /camera/annotations:message_encodi

Notes on schema:
* Generally the overall tagged component layout is nice and makes sense if I'm thinking about it from a topic perspective.
* The nesting under `TextLog` looks nice but it feels like we should get the same under `mcap.Channel` and `mcap.Schema` (ex. `/camera/image:metadata` -> `/camera/image:Channel:metadata`)
    * I believe this lack of unique naming also seems to cause a problem in our dataframe extraction. See below where `foxglove.RawImage` `data` and `mcap.Schema` `data` don't both appear to show up

In [17]:
for component in rec.schema():
    if isinstance(component, rr.dataframe.IndexColumnDescriptor):
        continue
    if "image" in component.entity_path and "data" in component.component:
        print(component)

Column name: /camera/image:metadata
	Entity path: /camera/image
	Archetype: rerun.mcap.Channel
	Component type: 
	Component: metadata
Column name: /camera/image:data
	Entity path: /camera/image
	Archetype: rerun.mcap.Schema
	Component type: 
	Component: data


In [27]:
view = rec.view(index="publish_time", contents="/camera/image/**")

In [29]:
# This looks like a bug because the mcap schema should be static but we query the publish time index above
view.schema()

Column name: /camera/image:frame_id
	Entity path: /camera/image
	Archetype: foxglove.RawImage
	Component type: 
	Component: frame_id
Column name: /camera/image:height
	Entity path: /camera/image
	Archetype: foxglove.RawImage
	Component type: 
	Component: height
Column name: /camera/image:step
	Entity path: /camera/image
	Archetype: foxglove.RawImage
	Component type: 
	Component: step
Column name: /camera/image:timestamp
	Entity path: /camera/image
	Archetype: foxglove.RawImage
	Component type: 
	Component: timestamp
Column name: /camera/image:width
	Entity path: /camera/image
	Archetype: foxglove.RawImage
	Component type: 
	Component: width
Column name: /camera/image:TextLog:level
	Entity path: /camera/image
	Archetype: rerun.archetypes.TextLog
	Component type: rerun.components.TextLogLevel
	Component: TextLog:level
Column name: /camera/image:TextLog:text
	Entity path: /camera/image
	Archetype: rerun.archetypes.TextLog
	Component type: rerun.components.Text
	Component: TextLog:text
Col

In [30]:
batches = view.select()

In [36]:
next_batch = next(iter(batches))
print(next_batch)

pyarrow.RecordBatch
log_time: timestamp[ns]
publish_time: timestamp[ns]
/camera/image:frame_id: list<item: string>
  child 0, item: string
/camera/image:height: list<item: uint32>
  child 0, item: uint32
/camera/image:step: list<item: uint32>
  child 0, item: uint32
/camera/image:timestamp: list<item: struct<seconds: int64, nanos: int32>>
  child 0, item: struct<seconds: int64, nanos: int32>
      child 0, seconds: int64
      child 1, nanos: int32
/camera/image:width: list<item: uint32>
  child 0, item: uint32
/camera/image:TextLog:level: list<item: string>
  child 0, item: string
/camera/image:TextLog:text: list<item: string>
  child 0, item: string
/camera/image:id: list<item: uint16>
  child 0, item: uint16
/camera/image:message_encoding: list<item: string>
  child 0, item: string
/camera/image:topic: list<item: string>
  child 0, item: string
/camera/image:data: list<item: binary>
  child 0, item: binary
/camera/image:encoding: list<item: string>
  child 0, item: string
/camera/im

In [38]:
# This looks like our static data
next_batch["/camera/image:data"].flatten()

[
  0AFF010A1F676F6F676C652F70726F746F6275662F74696D657374616D702E70726F746F120F676F6F676C652E70726F746F627566223B0A0954696D657374616D7012180A077365636F6E647318012001280352077365636F6E647312140A056E616E6F7318022001280552056E616E6F734285010A13636F6D2E676F6F676C652E70726F746F627566420E54696D657374616D7050726F746F50015A32676F6F676C652E676F6C616E672E6F72672F70726F746F6275662F74797065732F6B6E6F776E2F74696D657374616D707062F80101A20203475042AA021E476F6F676C652E50726F746F6275662E57656C6C4B6E6F776E5479706573620670726F746F330AE7010A17666F78676C6F76652F526177496D6167652E70726F746F1208666F78676C6F76651A1F676F6F676C652F70726F746F6275662F74696D657374616D702E70726F746F2298010A08526177496D616765122D0A0974696D657374616D7018012001280B321A2E676F6F676C652E70726F746F6275662E54696D657374616D7012100A086672616D655F6964180720012809120D0A057769647468180220012807120E0A0668656967687418032001280712100A08656E636F64696E67180420012809120C0A0473746570180520012807120C0A046461746118062001280C620670726F746F33
]

In [39]:
static_batches = rec.view(index=None, contents="/camera/image/**").select()

In [35]:
print(next(iter(static_batches)))

pyarrow.RecordBatch
log_time: timestamp[ns]
publish_time: timestamp[ns]
/camera/image:TextLog:level: list<item: string>
  child 0, item: string
/camera/image:TextLog:text: list<item: string>
  child 0, item: string
/camera/image:id: list<item: uint16>
  child 0, item: uint16
/camera/image:message_encoding: list<item: string>
  child 0, item: string
/camera/image:topic: list<item: string>
  child 0, item: string
/camera/image:data: list<item: binary>
  child 0, item: binary
/camera/image:encoding: list<item: string>
  child 0, item: string
/camera/image:name: list<item: string>
  child 0, item: string
----
log_time: [null]
publish_time: [null]
/camera/image:TextLog:level: [["WARN"]]
/camera/image:TextLog:text: [["Unsupported schema for channel: foxglove.RawImage"]]
/camera/image:id: [[1]]
/camera/image:message_encoding: [["protobuf"]]
/camera/image:topic: [["/camera/image"]]
/camera/image:data: [[0AFF010A1F676F6F676C652F70726F746F6275662F74696D657374616D702E70726F746F120F676F6F676C652E7